In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices

In [5]:
dta = pd.read_csv('../data/dayData.csv')

In [6]:
dta.describe()

,Unnamed: 0,household_key,DAY,QUANTITY,BASE_SPEND_AMT,NET_SPEND_AMT,LOY_CARD_DISC,COUPON_DISC,GET_EGGS,WEEK
count,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.0
mean,12899.000000,505.790402,101.607086,14.598752,42.941644,36.737954,-6.060541,-0.143149,0.146063,0.0
std,7447.096783,292.033469,58.799244,17.332974,51.432689,43.380989,8.682502,0.921875,0.367696,0.0
min,1.000000,1.000000,0.000000,1.000000,0.210000,0.210000,-208.020000,-43.280000,0.000000,0.0
25%,6450.000000,257.000000,51.000000,3.000000,10.060000,8.970000,-7.830000,0.000000,0.000000,0.0
50%,12899.000000,511.000000,102.000000,8.000000,24.190000,21.010000,-2.970000,0.000000,0.000000,0.0
75%,19348.000000,761.000000,152.000000,19.000000,55.730000,47.560000,-0.790000,0.000000,0.000000,0.0
max,25797.000000,1000.000000,204.000000,261.000000,816.790000,681.650000,0.000000,0.000000,3.000000,0.0


In [7]:
dta = dta.drop('Unnamed: 0', 1)

In [8]:
y, X = dmatrices('GET_EGGS ~ DAY + QUANTITY + BASE_SPEND_AMT + NET_SPEND_AMT + LOY_CARD_DISC + COUPON_DISC', dta, return_type="dataframe")

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
predicted = model.predict(X_test)
print(prediction)

NameError: name 'prediction' is not defined

In [12]:
probs = model.predict_proba(X_test)
print(probs)

[[  9.06108658e-01   9.06275074e-02   3.22083016e-03   4.30044867e-05]
 [  9.02909100e-01   9.25835670e-02   3.98556428e-03   5.21768985e-04]
 [  9.34581238e-01   6.19655506e-02   2.93718244e-03   5.16028743e-04]
 ..., 
 [  9.15034667e-01   8.09995977e-02   3.38595095e-03   5.79784829e-04]
 [  9.11912632e-01   8.47456278e-02   3.27385264e-03   6.78873830e-05]
 [  9.32026272e-01   6.48839839e-02   2.89278984e-03   1.96954347e-04]]


In [30]:
metrics.brier_score_loss(y_test, predicted)

0.00025839793281653745

In [14]:
print(y_train.sum(axis=0)[0])
print(y_train.shape[0])
freq = y_train.sum(axis=0)[0] / y_train.shape[0]
print(y_test.shape[0])
predicted = [0 for idx in range(0, y_test.shape[0])]

2633.0
18057
7740


In [16]:
y_test['GET_EGGS'].value_counts()

0.0    6643
1.0    1061
2.0      34
3.0       2
Name: GET_EGGS, dtype: int64

In [17]:
np.count_nonzero(predicted==1)

0

In [21]:
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
prediction2 = model2.predict(X_test)
print(prediction2)

[ 0.  0.  0. ...,  0.  0.  0.]


In [23]:
metrics.brier_score_loss(y_test, prediction2)

ValueError: y_prob contains values greater than 1.

,Intercept,DAY,QUANTITY,BASE_SPEND_AMT,NET_SPEND_AMT,LOY_CARD_DISC,COUPON_DISC
0,1.0,0.0,1.0,2.84,2.84,0.00,0.00
1,1.0,0.0,47.0,115.45,99.03,-15.42,-1.00
2,1.0,0.0,44.0,133.85,112.27,-21.58,0.00
3,1.0,0.0,15.0,34.33,29.50,-4.83,0.00
4,1.0,0.0,5.0,13.44,12.48,-0.96,0.00
5,1.0,0.0,11.0,35.22,31.37,-3.85,0.00
6,1.0,0.0,32.0,106.58,87.95,-18.63,0.00
7,1.0,0.0,5.0,18.93,17.15,-1.78,0.00
8,1.0,0.0,18.0,49.28,40.62,-8.66,0.00
9,1.0,0.0,1.0,1.59,1.29,-0.30,0.00


In [26]:
import xgboost as xgb

In [31]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
predictions = gbm.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
metrics.brier_score_loss(y_test, predictions)

0.01925064599483204